In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib notebook

In [ ]:
import datetime
from dataclasses import dataclass
import tqdm
import importlib

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from databroker import catalog

In [ ]:
# import bact_analysis
# import bact_analysis.bba.calc
# import bact_analysis.transverse.calc
# import bact_analysis.transverse.distorted_orbit
# import bact_analysis.transverse.process
import bact_analysis.utils.preprocess
import bact_analysis_bessyii.bba.preprocess_data
# import bact_math_utils

In [ ]:
from bact_math_utils.linear_fit import linear_fit_1d

# BBA Analysis

This notebook should be the center for selecting the required data and presenting the results.
All computation or data rearangement shall be performed in libraries

In [ ]:
datetime.datetime.now().strftime('%Y %m %d %H:%M')

## Data selection, check and load

Define the catalog database. In case of questions have a look at https://blueskyproject.io/databroker/

In [ ]:
catalog_name = 'datascc_bba_test'

Standard large database for HZB

In [ ]:
catalog_name = 'heavy'

In [ ]:
db = catalog[catalog_name]

In [ ]:
db[-1]

Define the universal id of the run

In [ ]:
uid = '6c3d31df-0714-4ea9-b7e0-64df3ff039c0'

Here an illustration how to search for measurements

* I first define the acceptable time range
* From this time range I use the one that have the nickname "bba" set

In [ ]:
from databroker.queries import TimeRange

In [ ]:
t_search =  db.search(TimeRange(since="2022-08-17", until="2022-08-20"))
possible_bba = t_search.search(dict(nickname="bba"))

l = []
for uid in possible_bba:
    run = db[uid]
    start = run.metadata["start"]    
    ts_start = datetime.datetime.fromtimestamp(int(start['time']))
    stop = run.metadata["stop"]
    # print(run.metadata['start'].keys())
    nickname = run.metadata['start']["nickname"]
    if not stop:
        # print(f'{uid} {nickname:20s} {ts_start} ----')
        continue
    ts_end = datetime.datetime.fromtimestamp(int(stop['time']))
    # print(f'{uid} {nickname:20s} {ts_start} {ts_end}')
      
    l.append((uid, ts_start, ts_end))
    
    
def htmlify_table_entry(entry, *, label, newline=''):
    tmp = f' </{label}>{newline}<{label}> '.join([str(tmp) for tmp in entry])
    return f'<{label}> {tmp} </{label}>{newline}'

txt = htmlify_table_entry([htmlify_table_entry(entry, label='td') for entry in l[:4]][::-1], label='tr', newline='\n')
print(txt)
print([e[0] for e in l[:4][::-1]])

In [ ]:
uid = '20e55c8b-6804-4f5f-bf22-a5a6a8521e41'
# uid = '874631e7-fec9-4666-a112-8dfa2e42545c'

In [ ]:
# uid = 'e5c25993-feab-4820-b4c5-b6d27e97942a'

In [ ]:
uid = '22354cec-864b-4f38-ad46-a9641d07d1ac'
uid = '1e6ec7f3-44a5-4e5c-a798-1d7ce12aafe1'
uid = "eb89753c-5388-4ebb-a992-fc25b478acd8"


For measurement see elog entry http://elog-v2.trs.bessy.de:8080/Machine+Devel.,+Comm./1971

In [ ]:
uid = 'b226943c-1941-46ce-bc35-0530ea6e276c'
uid = 'e0aef7b8-f57e-4594-9618-70d01aaa65a7'
uid = 'e60215ff-62ea-4d3b-a968-f6b0d9d9ee9d'
uid = 'fa22af2e-0398-41eb-94b9-e9b957ba4f31'

In [ ]:
 run = db[uid]

### loading data

In [ ]:
descriptor, = run.primary.metadata['descriptors']
descriptor;

In [ ]:
configuration = descriptor['configuration']
dt_configuration = configuration['dt']
list(configuration.keys()), list(dt_configuration.keys())

Loading dask seems to be the fastest approach

In [ ]:
start = datetime.datetime.now()
all_data_ = run.primary.to_dask()
end = datetime.datetime.now()

end - start

Now load all data

In [ ]:
for name, item in tqdm.tqdm(all_data_.items(), total=len(all_data_.variables)):
    item.load()

The bpm names should be stored as configuration values: this is not the case for old data

In [ ]:
bpm_names = all_data_.dt_bpm_waveform_names.values[0]

### Collapsing the number of different dimensions

Replace them with known lablled alternatives

In [ ]:
bpm_dims = bact_analysis_bessyii.bba.preprocess_data.replaceable_dims_bpm(
    all_data_, prefix="dt_", expected_length=len(bpm_names)
)

### Derive info on measurement

When were magnets strength was switched or magnet was reselected

In [ ]:
muxer_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value(all_data_.dt_mux_power_converter_setpoint)
muxer_pc_current_change.name = "muxer_pc_current_change" 
muxer_or_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value_pairs(all_data_.dt_mux_power_converter_setpoint, all_data_.dt_mux_selector_selected)
muxer_or_pc_current_change.name = "muxer_or_pc_current_change" 

### Combine all info to new xarray 

In [ ]:
replace_dims = {dim : 'bpm' for dim in bpm_dims}
# replace_dims.update({dim : 'pos' for dim in beam_dims})
all_data = all_data_.rename(replace_dims).assign_coords(bpm=list(bpm_names))
all_data;

In [ ]:
preprocessed = xr.merge([all_data, muxer_pc_current_change, muxer_or_pc_current_change])
preprocessed;

### Make data selectable per magnet

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)
importlib.reload(bact_analysis.transverse.calc)
importlib.reload(bact_analysis_bessyii.bba.preprocess_data)

In [ ]:
type(preprocessed)

In [ ]:
rearranged = xr.concat(
    bact_analysis.utils.preprocess.reorder_by_groups(
        preprocessed,
        preprocessed.groupby(preprocessed.dt_mux_selector_selected),
        reordered_dim="name",
        dim_sel="time",
        new_indices_dim="step",
    ),
    dim="name",
)

In [ ]:
preprocessed.dt_mr_tune_fb_vert_readback

In [ ]:
preprocessed.dt_mr_tune_fb_hor_readback

In [ ]:
tune_data = xr.merge([rearranged.dt_mr_tune_fb_hor_readback, rearranged.dt_mr_tune_fb_vert_readback, rearranged.dt_mux_power_converter_setpoint, rearranged.dt_mux_power_converter_readback])

In [ ]:
tune_sel = tune_data.sel(name="Q2M1T1R")

In [ ]:
@dataclass
class TuneFitResult:
    x: np.ndarray
    std: np.ndarray
    
def fit_tune_shift(dI, tune):
    x, std =  linear_fit_1d(dI, tune)
    return TuneFitResult(x=x, std=std)

In [ ]:
res = fit_tune_shift(tune_sel.dt_mux_power_converter_setpoint, tune_sel.dt_mr_tune_fb_hor_readback)

In [ ]:
res

In [ ]:
x = np.linspace(-2.2, 2.2)
t_f = res.x[0] * x
y = res.x[1] + t_f
t_f_m = tune_sel.dt_mr_tune_fb_hor_readback  - res.x[1]

In [ ]:
fig, axes = plt.subplots(1, 2, num=100, figsize=[12, 6])
ax1, ax2= axes
line, = ax1.plot(tune_sel.dt_mux_power_converter_setpoint, tune_sel.dt_mr_tune_fb_hor_readback, 'x')
ax1.plot(x, y, '-', color=line.get_color())
ax1.set_xlabel(r"\Delta I [A]")
ax1.set_ylabel(r"f [kHz]")
ax2.plot(tune_sel.dt_mux_power_converter_setpoint, t_f_m, 'x', color=line.get_color())
ax2.plot(x, t_f, '-', color=line.get_color())
ax2.set_xlabel(r"\Delta I [A]")
ax2.set_ylabel(r"\Delta t [kHz]")


In [ ]:
all_data.dt_mr_tune_fb_hor_readback

In [ ]:
# measurement_vars = dict(dt='x_pos', dt_beam_orbit_y='y_pos', dt_mux_power_converter_setpoint='excitation')
measurement_vars = dict(
    dt_bpm_waveform_x_pos="x_pos",
    dt_bpm_waveform_y_pos="y_pos",
    dt_mr_tune_fb_hor_readback="x_tune",
    dt_mr_tune_fb_vert_readback="y_tune",
    dt_mux_power_converter_setpoint="excitation",
)
redm4proc = (
    rearranged[list(measurement_vars.keys())]
    .rename_vars(**measurement_vars)
    .sel(bpm=bpm_names)
)
redm4proc;

## Processing data

## Preparing for calculation

In [ ]:
redm4proc

## Fitting data

In [ ]:
def fit_tune_shift_all(ds, name):
    x = fit_tune_shift(ds.excitation, ds.x_tune)
    y = fit_tune_shift(ds.excitation, ds.y_tune)


    ndx = xr.DataArray(
        name="x",
        data=[[x.x, x.std]],
        dims=["name", "res", "coeff"],
        coords=[[name], ["val", "std"], ["slope", "intercept"]],
    )
    
    ndy = xr.DataArray(
        name="y",
        data=[[y.x, y.std]],
        dims=["name", "res", "coeff"],
        coords=[[name],["val", "std"], ["slope", "intercept"]],
    )
    
    nds = xr.merge([ndx, ndy])
    return nds

In [ ]:
tune_fits = xr.concat(
    [
        fit_tune_shift_all(redm4proc.sel(name=name), name)
        for name in redm4proc.coords["name"].values
    ],
    dim="name",
)
# tune_fits.metadata['uid'] = uid

In [ ]:
def adjust_tick(tick):
    tick.set_fontsize("small")
    tick.set_rotation(45)
    tick.set_horizontalalignment("right")
    tick.set_verticalalignment("top")


for fignum, family_name in enumerate(["Q4", "Q2", "Q3", "Q5", "Q1"]):
    sel = tune_fits.isel(
        name=[
            name[: len(family_name)] == family_name
            for name in tune_fits.coords["name"].values
        ]
    )
    fig, axes = plt.subplots(2, 1, num=fignum, sharex=True, figsize=[16, 6])
    ax_x, ax_y = axes
    names = sel.x.coords["name"].values
    line, x_err, y_err = ax_x.errorbar(
        names,
        sel.x.sel(res="val", coeff="slope"),
        yerr=sel.x.sel(res="std", coeff="slope"),
        linestyle="",
    )
    ax_y.errorbar(
        names,
        sel.y.sel(res="val", coeff="slope"),
        yerr=sel.y.sel(res="std", coeff="slope"),
        linestyle="",
        color=line.get_color()
    )

    [adjust_tick(tick) for tick in ax_y.get_xmajorticklabels()];